In [2]:

import sklearn as SK
import lightgbm as lgb
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import *
from skopt import BayesSearchCV
import numpy as np
import pandas as pd
import scipy.stats
from math import sqrt
import os 
import io
from tensorboard.plugins.hparams import api as hp
from utils import utils

commons = utils.Commons()
ml_helper = utils.ML_Helper()
modelGen = utils.Model_Generator()
Bayes = utils.BayesSearch_Helper()
ml_helper.model_type = ml_helper.Regression # Always set this variable to the type of model you are building
Bayes.model_type = ml_helper.model_type
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "-1"


In [ ]:
#Inport train, validation and test sets
# Available Models are (LGBM,SVM,RF)

FOLD = 0 # 0-5 -4

GLOBAL_NUMBER_OF_TASKS = 8
PATH = ".\data\Eufrasia\splits\Random"

TASK_START = 2
LOCAL_NUM_TASKS = 1
SMILES_COL = 'SMILES'
SPLITS = ["train","val","test"]

#USE_DEFAULT_PARAMS = True

RADIUS = 2 #diameter 4
FP_SIZE = 2048  #bit string size
FEAT = False #used when you consider pharmacophoric features
#BUILD_PARAMS = {"fp_size":FP_SIZE,"radius":RADIUS,"feat":FEAT,"path":PATH,"global_tasks":GLOBAL_NUMBER_OF_TASKS,"smiles_col":SMILES_COL,"task_start":TASK_START,"local_num_tasks":LOCAL_NUM_TASKS,"which_splits":SPLITS,"fold":FOLD,"useDefaultParams":USE_DEFAULT_PARAMS,"selected_model":ml_helper.model_type}


#This is an example of how to use the functions, you can change non constant parameters to fit your needs
SELECTED_MODELS = ["lgbm","rf","svm"]
for SELECTED_MODEL in SELECTED_MODELS:
    for (df,y_data,smiles),which_split in commons.load_dataset_list(PATH, GLOBAL_NUMBER_OF_TASKS, SMILES_COL, TASK_START, LOCAL_NUM_TASKS, SPLITS, FOLD):
        x_data,y_data = commons.setML_FPs(smiles,y_data,FP_SIZE, RADIUS, FEAT)
        print(x_data.shape,y_data.shape)
        text,df = Bayes.get_Stats(SELECTED_MODEL,x_data,y_data)
        print(text)
        df["Selected Model"] = SELECTED_MODEL
        df["Split"] = which_split
        df["Fold"] = FOLD
        df.to_csv(f"results_for_{SELECTED_MODEL}.csv",index=False)
    # all_df = None
    # for df,split in buildThenFitThenGetStats(**BUILD_PARAMS):
    #     all_df = pd.concat([all_df,df],ignore_index=True)
    # all_df.to_csv(f"results_for_{SELECTED_MODEL}.csv",index=False)

In [ ]:
# calculate ECFP (defaut) fingerprints using RDKit
# RADIUS = 2 #diameter 4
# FP_SIZE = 2048  #bit string size
# FEAT = False #used when you consider pharmacophoric features
# def assign_fingerprints(smiles,fp_size,radius,feat):
#     return commons.assing_fp(smiles,fp_size,radius,feat)
# X_train = commons.assing_fp(train_smiles,FP_SIZE,RADIUS,FEAT)
# X_test = commons.assing_fp(test_smiles,FP_SIZE,RADIUS,FEAT)

# y_train = y_train.ravel()
# y_train = np.array(y_train).astype(int)
# y_test = y_test.ravel()
# y_test = np.array(y_test).astype(int)


In [ ]:

# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# scorer = make_scorer(mean_squared_error)

# build_model = model_generator.Models[SELECTED_MODEL]
# CLASSIFIER = build_model["classifier"]
# PARAMS = build_model["params"]

# log-uniform: understand as search over p = exp(x) by varying x

# best_model = BayesSearchCV(CLASSIFIER,
#     PARAMS,
#     n_iter=1, # Number of parameter settings that are sampled
#     cv=cv,
#     scoring = scorer,
#     refit = True, # Refit the best estimator with the entire dataset.
#     random_state=42,
#     n_jobs = -1
# )

# best_model.fit(X_train, y_train)

# print("Best parameters: %s" % best_model.best_params_)

In [ ]:
#RF best parameters: {'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 140}
#SVM best parameters: {'C': 0.001, 'gamma': 0.001, 'kernel': 'rbf'}
#LGBM best parameters: {'learning_rate': 0.04690935629679825, 'max_depth': 8, 'n_estimators': 47, 'num_leaves': 5, 'subsample': 0.23402958965378692}
# model = CLASSIFIER.set_params(**best_model.best_params_)
# model.probability = True
# model.fit(X_train, y_train)


In [ ]:
#text = ml_helper.get_ML_StatsForNSplits(model,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)

In [ ]:
# with open(f"results_for {SELECTED_MODEL}.txt", "a+") as f:
#     in_this_task = training[task].replace("train","").replace(path,"").replace(".csv","").replace("/","").replace(f"__{FOLD}","")
#     f.write(f"{in_this_task} Fold {FOLD}:\n {text}\r")